In [93]:
import pandas as pd
import json

In [94]:

def load_json_from_path(file_path):
    """
    Load JSON data from a file path.

    Parameters:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The JSON data loaded from the file.
    """
    with open(file_path, 'r', encoding="utf8") as file:
        json_data = json.load(file)
    return json_data



In [95]:
a1 = load_json_from_path('./data/article1.json')

In [96]:
a1

[{'title': 'Petro y la lucha de clases / Columna de Felipe López Caballero',
  'url': 'https://www.eltiempo.com/politica/gobierno/petro-y-la-lucha-de-clases-columna-de-felipe-lopez-caballero-3333631',
  'content': ['La explicación está en la narrativa que Petro y sus redes sociales han creado. Según esta: 1) Colombia ha estado gobernada durante 200 años por unas élites corruptas interesadas solamente en enriquecerse y perpetuarse en el poder.',
   '2) Con Petro llegaría el primer presidente de izquierda cuya preocupación son los pobres y la justicia social.',
   '3) Los dueños de Colombia no estarían dispuestos a ceder sus privilegios ante un exguerrillero que busca el cambio.',
   'Esa narrativa es simplista, efectista y dañina. Para comenzar, el problema no es que Petro sea un exguerrillero. Si el presidente fuera Antonio Navarro la economía estaría funcionando mejor y la polarización no estaría disparada. La mitad del estancamiento del PIB obedece a factores internacionales, pero la

In [97]:
import re
from unidecode import unidecode

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Remove accents
    text = unidecode(text)
    return text

def remove_stop_words(text, custom_stop_words):
    """
    Remove custom stop words from the input text.

    Args:
        text (str): Input text.
        custom_stop_words (list): List of custom stop words to be removed.

    Returns:
        str: Text with custom stop words removed.
    """
    stop_words = set(custom_stop_words)
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

In [98]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Sample text data
documents = [
    "Petro presidente de izquierda de colombia",
    "presidente de colombia es petro",
    "petro es presidente",
    "presidente colombia"
]
 
stop_words = ['de','es']
# Initialize CountVectorizer
vectorizer = CountVectorizer(stop_words=['de','es'])

# Fit and transform the documents
X = vectorizer.fit_transform(documents)


In [99]:
vectorizer.get_feature_names_out()

array(['colombia', 'izquierda', 'petro', 'presidente'], dtype=object)

In [100]:
def generate_pairs(text):
    words = text.split()
    pairs = []
    for i in range(len(words) - 1):
        pairs.append(words[i]+" "+words[i+1])
    return pairs

In [101]:
docs_pairs = []

# Generate pairs after removing stop words for each document
docs_pairs.extend([pair for d in documents for pair in generate_pairs(remove_stop_words(d, stop_words))])

print(docs_pairs)

['Petro presidente', 'presidente izquierda', 'izquierda colombia', 'presidente colombia', 'colombia petro', 'petro presidente', 'presidente colombia']


In [102]:
encoding = vectorizer.transform(docs_pairs).toarray()
co_ocurrences = encoding.T@encoding

names = vectorizer.get_feature_names_out().tolist()

np.fill_diagonal(co_ocurrences, 0)

pd.DataFrame(co_ocurrences, index=names, columns=names)

,colombia,izquierda,petro,presidente
colombia,0,1,1,2
izquierda,1,0,0,1
petro,1,0,0,2
presidente,2,1,2,0
